In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from scipy import ndimage, misc
from skimage.transform import resize, rescale

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

import os
import re

print(tf.__version__)


1.13.1


In [3]:
input_img = Input(shape=(256, 256, 3))

l1 = Conv2D(64, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(input_img)

l2 = Conv2D(64, 
            (3, 3),
            padding='same',
            activation='relu',
            activity_regularizer=regularizers.l1(10e-10)
           )(l1)

l3 = MaxPooling2D(padding='same')(l2)

l4 = Conv2D(128, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l3)

l5 = Conv2D(128, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l4)

l6 = MaxPooling2D(padding='same')(l5)

l7 = Conv2D(256, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l6)

encoder = Model(input_img, l7)


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
__________

In [5]:
input_img = Input(shape=(256, 256, 3))

l1 = Conv2D(64, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(input_img)

l2 = Conv2D(64, 
            (3, 3),
            padding='same',
            activation='relu',
            activity_regularizer=regularizers.l1(10e-10)
           )(l1)

l3 = MaxPooling2D(padding='same')(l2)

l4 = Conv2D(128, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l3)

l5 = Conv2D(128, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l4)

l6 = MaxPooling2D(padding='same')(l5)

l7 = Conv2D(256, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l6)

In [6]:
l8 = UpSampling2D()(l7)

l9 = Conv2D(128, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l8)

l10 = Conv2D(128, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l9)

l11 = add([l5, l10])

l12 = UpSampling2D()(l11)

l13 = Conv2D(64, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l12)

l14 = Conv2D(64, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l13)

l15 = add([l14, l2])

decoder = Conv2D(3, 
            (3, 3), 
            padding='same', 
            activation='relu', 
            activity_regularizer=regularizers.l1(10e-10)
           )(l15)


autoencoder = Model(input_img, decoder)

In [7]:
autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_5[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (

In [8]:
autoencoder.compile(
    optimizer='adadelta',
    loss='mean_squared_error'
)

Instructions for updating:
Use tf.cast instead.
